# Flat-<i>k</i> Gaussian <i>P(k)</i>

Bobby Pascua and the HERA Validation Team

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Flat-k-Gaussian-P(k)" data-toc-modified-id="Flat-k-Gaussian-P(k)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Flat-<i>k</i> Gaussian <i>P(k)</i></a></span><ul class="toc-item"><li><span><a href="#Setup-and-Metadata" data-toc-modified-id="Setup-and-Metadata-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Setup and Metadata</a></span><ul class="toc-item"><li><span><a href="#Abstract" data-toc-modified-id="Abstract-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>Abstract</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#Description" data-toc-modified-id="Description-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Summary</a></span></li><li><span><a href="#Software" data-toc-modified-id="Software-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Software</a></span></li><li><span><a href="#Data" data-toc-modified-id="Data-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>Data</a></span></li></ul></li><li><span><a href="#Data-Retrieval" data-toc-modified-id="Data-Retrieval-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Data Retrieval</a></span><ul class="toc-item"><li><span><a href="#Power-Spectrum-Retrieval" data-toc-modified-id="Power-Spectrum-Retrieval-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Power Spectrum Retrieval</a></span></li><li><span><a href="#Power-Spectrum-Amplitude-Expectation" data-toc-modified-id="Power-Spectrum-Amplitude-Expectation-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Power Spectrum Amplitude Expectation</a></span></li></ul></li><li><span><a href="#Data-Visualization" data-toc-modified-id="Data-Visualization-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Data Visualization</a></span><ul class="toc-item"><li><span><a href="#Ensemble-Averaged-Power-Spectrum" data-toc-modified-id="Ensemble-Averaged-Power-Spectrum-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Ensemble-Averaged Power Spectrum</a></span></li><li><span><a href="#Histograms" data-toc-modified-id="Histograms-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Histograms</a></span></li></ul></li></ul></li></ul></div>

## Setup and Metadata

### Abstract

This notebook validates the `hera_pspec.pspecdata` pipeline's ability to recover a flat-<i>k</i> Gaussian power spectrum from an ensemble of noiseless simulations. We extract the power spectrum estimate from each simulation using the `hera_pspec.pspecdata` pipeline and perform an ensemble average on the resulting spectra. We find agreement between the measured and predicted power spectrum amplitude at the 0.15% level. Additionally, we histogram the measured powers from the full ensemble and find that the distribution is consistent with an exponential distribution, with the scale parameter given by the expected power spectrum amplitude.

### Imports

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import glob
from datetime import datetime

import hera_pspec as hp
from pyuvdata import UVData
import pyuvdata
import os

ModuleNotFoundError: No module named 'hera_pspec'

In [1]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});
MathJax.Hub.Queue(
  ["resetEquationNumbers", MathJax.InputJax.TeX],
  ["PreProcess", MathJax.Hub],
  ["Reprocess", MathJax.Hub]
);

<IPython.core.display.Javascript object>

### Description

Last executed: {{str(datetime.now())}}

- **Major Step Description:** Simulate sky-locked, white-spectrum Gaussian field and estimate power spectrum with `hera_pspec.pspecdata`
- **Minor Variation Description:** Generate flat-k, sky-locked Gaussian field and estimate power spectrum from an ensemble average.
- **Pipelines Tested:** `hera_pspec.pspecdata`
- **Criteria**:
  1. Ensemble averaged power spectrum agrees with analytic prediction to within the predicted mean standard error.
  

### Summary

The results of this validation test, in reference to the outlined criteria, are
1. Test passed; ensemble averaged power spectrum estimate agrees with analytic expectation to 0.15%

The ensemble of power spectrum estimates was histogrammed and the result nearly perfectly follows an exponential distribution with scale parameter set by the expected power spectrum amplitude.

### Software

HERA software used in this validation test, with associate git commit hash:

* ``hera_pspec``: ``{{hp.version.git_hash}}``
* ``pyuvdata``: ``{{pyuvdata.version.git_hash}}``

Versions of other software used in this validation test:

* ``numpy``: ``v{{np.version.version}}``
* ``matplotlib``: ``v2.2.3``

### Data

The following paths reflect the exact locations of all data used in this test:

In [3]:
path0 = "/lustre/aoc/projects/hera/alanman/eor_sky_sim/"
path1 = "/lustre/aoc/projects/hera/Validation/test-0.1.0/Spectra/"

## Data Retrieval



### Power Spectrum Retrieval

This section offers two methods of retrieving the ensemble of power spectra: the first is by loading in the pre-processed power spectra, while the second involves obtaining the power spectra from the visibility data.

In [9]:
# try to make a glob of the pre-processed files, make glob of sim files if not
try:
    dfiles = sorted(glob.glob('{}eor*.npz'.format(path1)))
    data = np.load(dfiles[0])['arr_0']
    prepro = True
except:
    dfiles = sorted(glob.glob('{}*24.*.uvh5'.format(path0)))
    prepro = False

# either way, we want to make a histogram, so let's define the bin edges
bins = np.linspace(0, 1e7, 49)

# if pre-processed files can't be found, then we need to process the simulation files
if not prepro:
    # process files
    # this may be tricky, due to memory considerations. a single power spectrum data
    # array occupies roughly 145 MB. if we only want to look at the ensemble averaged
    # spectrum for one choice of taper, then this will certainly work with 32 GB of
    # memory allocated to the process, but will be cutting it close and may throw a
    # memory error if only 16 GB of memory is allocated to the process.
    # this can be done in a much less memory-intensive way if we don't care about the
    # individual power spectrum estimates, and only want the ensemble average. 
    # either way, this task is best done in parallel, but it's not exactly clear how
    # to implement parallelization of this task.
    # note that if this task is completed without using a parallelized algorithm,
    # then it will take roughly 24 hours to complete on the NRAO cluster
    
    
    pass # just leaving this here until I fill in the rest of this block
else:
    # get the ensemble average and fill out the histogram
    
    # first, make an array of zeros for the ensemble average
    avg_spec = np.zeros(data.shape, dtype='complex128')
    
    # since we were able to load in the pre-processed files, let's make
    # histograms for spectra computed with and without a taper
    hist = np.zeros( (2, 3,len(bins)-1), dtype='float64') # no taper
    
    # we'll need to loop over data files
    for f in dfiles:
        # load in the data
        data = np.load(f)['arr_0']
        
        # add it to the average spectrum, to be divided by the number of realizations
        # later on in the script
        avg_spec += data
        
        # update the histograms, but do this on a per-baseline basis
        # we need to take the abs of the data first for histogramming
        data = np.abs(data)
        # from the README, axis-1 corresponds to baselines
        for j in range(data.shape[1]):
            # axis-0 corresponds to taper choice
            # data[0] -> bh-taper
            # data[1] -> no taper
            hist[0,j] += np.histogram(data[1,j], bins=bins)[0]
            hist[1,j] += np.histogram(data[0,j], bins=bins)[0]
            
    # we're done with our loops, so now divide the average spectrum array by the 
    # number of realizations
    avg_spec /= len(dfiles)
    
    # let's normalize the histograms while we're at it
    for j in range(hist.shape[1]):
        hist[0,j] /= hist[0,j].sum()
        hist[1,j] /= hist[1,j].sum()

### Power Spectrum Amplitude Expectation

In this section, we determine the expected amplitude of the power spectrum. The expected amplitude is given by $P_0 = \sigma^2\Delta\nu\Delta\Omega X^2Y$, where $\sigma^2$ is the variance of the sky-locked, spectrally-flat Gaussian distribution of 21-cm brightness temperature fluctuations; $\Delta\nu$ is the size of a frequency channel; $\Delta\Omega$ is the size of a pixel in the HEALPix map used for the simulation, given by $\Delta\Omega = 4\pi/(12N_{side}^2)$; $X$ and $Y$ are cosmological scalars, used in the usual sense, evaluated at the redshift of the 21-cm line at band-center.

In [ ]:
# get a glob of simulation files and choose one at random
dfile = glob.glob('{}*24*.uvh5'.format(path0))[0]

# instantiate a UVData object and read the data file
uvd = UVData()
uvd.read_uvh5(dfile)

# get the parameters needed to compute the expected amplitude of the power spectrum
nside = uvd.extra_keywords[u'nside']
dOmega = 4*np.pi/(12*nside**2)                 # sr
skysig = uvd.extra_keywords[u'skysig'] * 1e3   # mK
df = np.diff(np.unique(uvd.freq_array)).mean() # Hz
f0 = 1420405751.767         # HI rest-frequency, Hz
z = f0/np.median(np.unique(uvd.freq_array)) - 1 # redshift of HI line at band-center
X2Y = hp.conversions.Cosmo_Conversions().X2Y(z) # (h^-1 Mpc)^3 sr^-1 Hz^-1

# calculate expected power spectrum amplitude
P0 = skysig**2*df*dOmega*X2Y                    # mK^2 h^-3 Mpc^3

## Data Visualization

In this section, we visualize the data collected in the previous section. We compare the ensemble-averaged power spectra to the expected power spectrum amplitude on a per-baseline basis, and we also examine the result averaged over baselines. We conclude the section by displaying the histograms calculated above, again on a per-baseline basis.

### Ensemble-Averaged Power Spectrum

In [ ]:
# calculate the difference in the beam-squared integral (between the simulation and the
# analytic result for a Gaussian beam), and get the delays for the power spectrum

# define the cosmology
cosmo = hp.conversions.Cosmo_Conversions()

# get the beam fwhm
bm_fwhm = np.radians(uvd.extra_keywords[u'bm_fwhm'])

# make the beam object
uvb = hp.PSpecBeamGauss(fwhm=bm_fwhm, beam_freqs=np.unique(uvd.freq_array), cosmo=cosmo)

# get the beam-squared integral from the extra keywords and calculate it with hera_pspec
bsq_int = uvd.extra_keywords[u'bsq_int']
bsq_int_hp = uvb.power_beam_sq_int(pol='pI')

# get the difference in the beam-squared integral
# hera_pspec uses the analytic result, so we'll take that as the reference
# for the correct value
bsq_int_diff = bsq_int_hp - bsq_int
bsq_int_diff_pct = 100*bsq_int_diff/bsq_int_hp

# use hera_pspec.utils.get_delays to get the delays
dlys = hp.utils.get_delays(np.unique(uvd.freq_array))

# make a list of baselines
bls = [(0,11), (0,12), (11,12)]

In [ ]:
# average the results over time (axis-2) and plot the resulting delay spectrum for
# each baseline, as well as the baseline-averaged result. for bonus points, include
# a smaller figure with each plot showing the residual?
time_avg_spec = avg_spec.mean(axis=2)
bl_avg_spec = time_avg_spec.mean(axis=1)

colors = ['teal', 'magenta', 'gold']
tapers = ['Blackman-Harris', 'No']

fig = plt.figure(figsize=(12,8))
axes = fig.subplots(2,1)
for j, taper in enumerate(tapers):
    # label axes
    axes[j].set_xlabel('Delay [ns]', fontsize=12)
    axes[j].set_ylabel(r'Power [mK$^2$ $h^{-3}$ Mpc$^3$]', fontsize=12)
    axes[j].set_title('Ensemble Averaged Power Spectrum Estimate Using {} Taper'.format(taper), fontsize=12)

    # plot spectra for each baseline
    for k, bl in enumerate(bls):
        axes[j].plot(dlys, time_avg_spec[j,k], color=colors[k], alpha=0.7, label=bl)
    
    # plot baseline-averaged spectrum
    axes[j].plot(dlys, bl_avg_spec[j], color='steelgrey', alpha=0.7, label='Baseline Average')
    
    # plot expected amplitude
    axes[j].axhline(P0, ls='--', color='k', alpha=0.8, label='Expected Value')
    
    # make a legend
    axes[j].legend(ncol=2)

# show the plot
plt.tight_layout()
plt.show()

**Figure x |** something

In [ ]:
# show the discrepancy in the results numerically

# calculate estimated power spectrum amplitudes
avg_spec_amps = time_avg_spec.mean(axis=-1)

# loop over tapers
for j, taper in enumerate(tapers):
    # loop over baselines
    for k, bl in enumerate(bls):
        disc = 100*(P0 - avg_spec_amps[j,k])/P0
        print('\nPercent discrepancy for baseline {} with {} taper: {}%'.format(bl, taper, disc))
    
    # show discrepancy for baseline-averaged results
    disc = 100*(P0 - avg_spec_amps[j].mean())/P0
    print('\nPercent discrepancy for baseline-average with {} taper: {}%'.format(taper, disc))
    
# for reference, show discrepancy in beam-squared integral
print('\nDiscrepancy in beam-squared integral: {}%'.format(bsq_int_diff_pct))

### Histograms

In [ ]:
# calculate an exponential distribution using the bins from earlier and the power
# spectrum expected amplitude. normalize it by using .sum() on the array. plot the
# histograms on top of the exponential distribution.
exp_dist = np.exp(-powers/P0)
normed_exp = exp_dist/exp_dist.sum()

# make plots
fig = plt.figure(figsize=(15,15))
axes = fig.subplots(3,2)
for j, bl in enumerate(bls):
    for k, taper in enumerate(tapers):
        # label axes
        axes[j,k].set_xlabel(r'Power [mK$^2$ $h^{-3}$ Mpc$^3$]', fontsize=12)
        axes[j,k].set_ylabel(r'$N/N_\mathrm{tot}$', fontsize=12)
        axes[j,k].set_title('Baseline: {}; Taper: {}'.format(bl, taper))
        
        # plot exponential distribution first
        axes[j,k].plot(powers, normed_exp, color='magenta', alpha=0.8, label='Expected Distribution')
        
        # plot measured distribution
        axes[j,k].scatter(powers, hist[k,j], color='k', marker='o', label='Measured Distribution')
        
        # add a legend
        axes[j,k].legend(loc='upper right')
        
# show the figure
plt.tight_layout()
plt.show()

**Figure x |** something